## Setup

In [ ]:
# Install packages
%pip install -qU pinecone-client \
    pinecone-plugin-assistant \
    python-dotenv

In [ ]:
# Load environment variables
import os
from dotenv import load_dotenv
load_dotenv(override=True)

api_key = os.getenv("PINECONE_API_KEY")
assistant_name = os.getenv("PINECONE_ASSISTANT_NAME")

## Setup Pinecone Assistant

In [ ]:
# Connect to Pinecone
from pinecone import Pinecone
from tqdm.autonotebook import tqdm

pc = Pinecone(api_key=api_key)
metadata = {}

*Optional: set metadata for your assistant by uncommenting and updating the next cell.*

In [ ]:
# metadata = {"author": "Jane Doe", "version": "1.0"}

### Create Assistant

In [ ]:
assistant = pc.assistant.create_assistant(
    assistant_name=assistant_name, 
    metadata=metadata, 
    timeout=30 # Wait 30 seconds for assistant operation to complete.
)

In [ ]:
import time

# Wait for the assistant to be ready
status = pc.assistant.list_assistants()[0].status
while status != 'Ready':
    time.sleep(1)

# Print the assistant details
pc.assistant.list_assistants()

### Upload files

To upload multiple files, run the cells in this section multiple times.

In [ ]:
import os

file_path = input("Enter the path to the file you want to upload: ")

# Get the file extension
_, file_extension = os.path.splitext(file_path)
   
# Convert the extension to lowercase for case-insensitive comparison
file_extension = file_extension.lower()
    
# Check if the file extension is .pdf or .txt
if file_extension not in ['.pdf', '.txt']:
    raise ValueError(f"Invalid file type. Expected .pdf or .txt, but got {file_extension}")

In [ ]:
import os

# Upload file
if os.path.exists(file_path):
    response = assistant.upload_file(
        file_path=file_path,
        timeout=None
    )
    print(f"Uploaded {file_path}")
    print("Response:", response)
else:
    print(f"File not found: {file_path}")

In [ ]:
# List all uploaded files once they have been processed
for file in assistant.list_files():
    while file.status != 'Available':
        time.sleep(1)
    print(file)

### Send a test query

In [ ]:
query = input("Enter a query to send to your Assistant: ")

In [ ]:
from pinecone_plugins.assistant.models.chat import Message

chat_context = [Message(content=query)]
response = assistant.chat_completions(messages=chat_context)
answer = response['choices'][0]['message']['content']
print(answer)